In [1]:
import talib
# import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5
import time

In [30]:
if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
login = 114781990
password = 'Mypassword$1234'
server = 'Exness-MT5Trial6'

mt5.login(login, password, server)

True

In [27]:
# For getting price at 40th second of 15 MIN

import datetime

# Get the current time
while True:
    current_time = datetime.datetime.now()

    # Check if the second value is equal to 40
    if current_time.second == 40:
        
        rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 1)

        df = pd.DataFrame(rate)


        df['date']=pd.to_datetime(df['time'], unit='s')

        print(df.head())

        break


         time     open     high      low    close  tick_volume  spread  \
0  1693839600  1.07839  1.07879  1.07832  1.07867          153       8   

   real_volume                date  
0            0 2023-09-04 15:00:00  


In [35]:
# Calculate Keltner's Channel

def get_kc( high, low, close, kc_lookback, multiplier, atr_lookback):
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift()))
    tr3 = pd.DataFrame(abs(low - close.shift()))

    # Calculate True Range as the element-wise maximum of the three components
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/atr_lookback).mean()

    kc_middle = close.ewm(span=kc_lookback).mean()
    kc_upper = kc_middle + multiplier * atr
    kc_lower = kc_middle - multiplier * atr

    return kc_middle, kc_upper, kc_lower

In [32]:
# # Getting data


# rate = mt5.copy_rates_from('EURUSDm', mt5.TIMEFRAME_M15, datetime.datetime.now(), 1000)
# df = pd.DataFrame(rate)


# df['date']=pd.to_datetime(df['time'], unit='s')

# df.to_excel('training.xlsx')

In [44]:
def check_prev_candle( element):

    current_index = df.loc[df.date ==  element.date].index[0]
    current_close = element.close
    current_candle = element.candle



    if current_index > 0:
        prev_index = current_index - 1 
    #           prev candle colour
        prev_candle = df.iloc[prev_index]['candle']

    #           current candle colour
        current_candle = df.iloc[current_index]['candle']

        if current_candle == 'G':
    #                 Both green current close must be higher
            if prev_candle == 'G' and df.iloc[current_index]['close'] > df.iloc[prev_index][ 'close']:
                return 1

    #                If last candle Red then entry candle must close above last candle open
            elif prev_candle == 'R' and df.iloc[current_index]['close'] > df.iloc[prev_index][ 'open']:
                return 1

            else:
                return 0

        elif current_candle == 'R':
    #                 Both red current close must be lower
            if prev_candle == 'R' and df.iloc[current_index]['close'] < df.iloc[prev_index][ 'close']:
                return 1

    #                If last candle green then entry candle must close below last candle open
            elif prev_candle == 'G' and df.iloc[current_index]['close'] < df.iloc[prev_index][ 'open']:
                return 1

            else:
                return 0

    #     How to check for the 1st candle?
    return 0


def candle( df):
    '''Return candle colour'''

    if df['close'] - df['open'] > 0:
        return 'G'

    else:
        return 'R'


In [45]:
# Making all indicators in training 
# Testing all logic on testing.xlsx

df = pd.read_excel('./training.xlsx', engine='openpyxl')

# Record the start time
start_time = datetime.datetime.now()

df['EMA_20'],df['upper_keltner'],df['lower_keltner'] = get_kc(df['high'], df['low'], df['close'], kc_lookback = 20, multiplier = 2.25, atr_lookback = 10)
df['candle'] = df.apply(candle, axis=1)
df['prev'] = df.apply(check_prev_candle, axis=1)
# Record the end time
end_time = datetime.datetime.now()

# Calculate the elapsed time
elapsed_time = end_time - start_time


print(f"Your function took {elapsed_time.total_seconds():.4f} seconds to run.")

Your function took 0.5474 seconds to run.


In [46]:
df.head()

,Unnamed: 0,date,open,high,low,close,EMA_20,upper_keltner,lower_keltner,candle,prev
0,0,2023-08-18 16:30:00,1.08731,1.08751,1.08709,1.08717,1.087170,1.088115,1.086225,R,0
1,1,2023-08-18 16:45:00,1.08717,1.08736,1.08677,1.08727,1.087222,1.088369,1.086076,G,0
2,2,2023-08-18 17:00:00,1.08726,1.08803,1.08726,1.08773,1.087409,1.088771,1.086046,G,1
3,3,2023-08-18 17:15:00,1.08773,1.08773,1.08680,1.08717,1.087340,1.088915,1.085765,R,1
4,4,2023-08-18 17:30:00,1.08717,1.08759,1.08707,1.08758,1.087398,1.088874,1.085922,G,0
